This is a mahine learning project to look into the cell phone consumer/user behavior

In [2]:
import pandas as pd
import os

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

In [3]:
df=pd.read_csv('/datasets/users_behavior.csv')

In [4]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


The number of observations is n=3214; the dataset is not super huge, so the decision is made to split the set into three: train, validate, and test according to the following ratio: 60:20:20

In [6]:
from sklearn.model_selection import train_test_split 
df_train, df_test = train_test_split (df, train_size=0.8, random_state=123 )
df_train, df_valid = train_test_split(df_train,test_size = 0.25, random_state=123)

In [7]:
df_train.info()
df_test.info()
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 1808 to 1525
Data columns (total 5 columns):
calls       1928 non-null float64
minutes     1928 non-null float64
messages    1928 non-null float64
mb_used     1928 non-null float64
is_ultra    1928 non-null int64
dtypes: float64(4), int64(1)
memory usage: 90.4 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 2377 to 1503
Data columns (total 5 columns):
calls       643 non-null float64
minutes     643 non-null float64
messages    643 non-null float64
mb_used     643 non-null float64
is_ultra    643 non-null int64
dtypes: float64(4), int64(1)
memory usage: 30.1 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 215 to 2338
Data columns (total 5 columns):
calls       643 non-null float64
minutes     643 non-null float64
messages    643 non-null float64
mb_used     643 non-null float64
is_ultra    643 non-null int64
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


In [8]:
features=df.drop(['is_ultra'], axis=1)
target=df['is_ultra']
train_features=df_train.drop(['is_ultra'], axis=1)
train_target=df_train['is_ultra']
valid_features=df_valid.drop(['is_ultra'], axis=1)
valid_target=df_valid['is_ultra']
test_features=df_test.drop(['is_ultra'], axis=1)
test_target=df_test['is_ultra']

In [9]:
print('Decision Tree')
for depth in range (1,5):
    tree_model=DecisionTreeClassifier(random_state=12345, max_depth=depth)
    tree_model.fit(train_features, train_target)
    tree_valid_predictions=tree_model.predict(valid_features)
    print('max_depth=', depth, ':', end="")
    print(accuracy_score(valid_target, tree_valid_predictions))
     
    

Decision Tree
max_depth= 1 :0.7356143079315708
max_depth= 2 :0.7651632970451011
max_depth= 3 :0.7838258164852255
max_depth= 4 :0.7822706065318819


I chose accuracy score for the classifier. It shows that the model accuracy is the highest when max depth is 3. It shows that the model is correct in predicting the correct answer about 78% of the time. Further, the ultimate depth of tree is 3 nodes. Extending the max depth to 4 does not yeld any better accuracy results.

In [10]:
print('Random Forest')
for i in range (10, 51,10):
    forest_model=RandomForestClassifier(n_estimators=i, max_depth=10, random_state=12345)
    forest_model.fit(train_features, train_target)
    #forest_valid_predictions=forest_model.predict(valid_features)
    #forest_mse=mean_squared_error(valid_target, forest_valid_predictions)
    #forest_rmse=forest_mse**0.5
    score=forest_model.score(train_features, train_target)
    print('n_estimator=', i,':', score)

Random Forest
n_estimator= 10 : 0.8884854771784232
n_estimator= 20 : 0.8921161825726142
n_estimator= 30 : 0.9004149377593361
n_estimator= 40 : 0.899896265560166
n_estimator= 50 : 0.899896265560166


Random Forest Classifer model was run with changing hyperparameters. The results show that the most accurate model is the random forest model with 30 trees.

In [22]:
print('Random Forest validation')
for i in range(10, 51, 10):
    forest_model=RandomForestClassifier(n_estimators=i, random_state=12345)
    forest_model.fit(train_features, train_target)
    score=forest_model.score(valid_features, valid_target)
    print('n_estimator=',i, score)

Random Forest validation
n_estimator= 10 0.7838258164852255
n_estimator= 20 0.776049766718507
n_estimator= 30 0.7853810264385692
n_estimator= 40 0.7838258164852255
n_estimator= 50 0.7853810264385692


In [23]:
model=LogisticRegression(random_state=12345)
model.fit(train_features, train_target)
model.score(valid_features, valid_target)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.702954898911353

Random forest regression seems to be the most accurate.

In [24]:
test_model=RandomForestClassifier(n_estimators=30, random_state=12345)
test_model.fit(train_features, train_target)
score_test=test_model.score(test_features, test_target)
print('test_random_forest', score_test)

test_random_forest 0.7978227060653188


Scoring the best model with the help of the test set shows very a slightly improved degree of accuracy. The random forest model explains with about 80% of accuracy.